In [25]:
from typing import List, Tuple
import numpy as np

Sequence = List[str]
Pattern = List[str]
PatternSequenceTuple = Tuple[Pattern, List[Sequence]]

In [26]:
def minDL(sequences: List[Sequence]) -> List[Pattern]:
	output: List[PatternSequenceTuple] = [(seq, [seq]) for seq in sequences]
	queue: List[float, PatternSequenceTuple, PatternSequenceTuple, PatternSequenceTuple] = []
	for seq in output:
		for otherSeq in output:
			if seq != otherSeq:
				delta, pattern = merge(seq, otherSeq)
				if delta > 0:
					queue.append((delta, pattern, seq, otherSeq))
	queue.sort(key=lambda val: val[0], reverse=True)
	print('Initialization phase over')
	while len(queue) > 0:
		delta, pattern, seq1, seq2 = queue.pop(0)
		# print('Queue length: ', len(queue))
		# print('Creating pattern: ', pattern[0], 'from sequences: ', seq1[0], 'and', seq2[0])

		queue = [x for x in queue if seq1 not in [x[2], x[3]] and seq2 not in [x[2], x[3]]]
		output = [x for x in output if x != seq1 and x != seq2]
		
		for otherSeq in output:
			new_delta, new_pattern = merge(otherSeq, pattern)
			if new_delta > 0:
				queue.append((new_delta, new_pattern, otherSeq, pattern))
				queue.sort(key=lambda val: val[0], reverse=True)
		output.append(pattern)
		# print(seq1, seq2, pattern)
	print('Queue is empty')
	return output

In [27]:
def merge(seq1: PatternSequenceTuple, seq2: PatternSequenceTuple) -> Tuple[float, PatternSequenceTuple]:
    alpha = 1
    lamda = 0
    pattern = longestCommonSubsquence(seq1[0], seq2[0])
    # get elements that are not in pattern, with position
    
    candidates = [x for x in set(seq1[0] + seq2[0]) if x not in pattern]
    candidates.sort(key=lambda x: sortCandidates(x, seq1, seq2), reverse=True)
    delta = -1
    
    # print('Merging sequences: ', seq1[0], 'and', seq2[0])
    # print('with LCS: ', pattern)
    # print('Candidates: ', candidates)
    for candidate in candidates:
        inner_delta = -1
        inner_pattern = pattern
        for position in range(0, len(pattern) + 1):
            pattern_with_candidate = pattern[:position] + [candidate] + pattern[position:]
            # print('Trying pattern: ', pattern_with_candidate)
            # print(len(seq1[0]) + len(seq2[0]) - len(pattern_with_candidate))
            # print(alpha * sum([edits(sequence, seq1[0]) for sequence in seq1[1]]))
            # print(alpha * sum([edits(sequence, seq2[0]) for sequence in seq2[1]]))
            # print(alpha * sum([edits(sequence, pattern_with_candidate) for sequence in seq1[1] + seq2[1]]))
            delta_tag: float = (len(seq1[0]) + len(seq2[0]) - len(pattern_with_candidate) 
                + alpha * sum([edits(sequence, seq1[0]) for sequence in seq1[1]]) 
                + alpha * sum([edits(sequence, seq2[0]) for sequence in seq2[1]])
                - alpha * sum([edits(sequence, pattern_with_candidate) for sequence in seq1[1] + seq2[1]])
                + lamda)
            # print('Delta tag: ', delta_tag)
            # print('Inner Delta: ', inner_delta)
            if delta_tag < 0 or delta_tag < inner_delta:
                # print("continue")
                continue
            else:
                # print('New delta: ', delta_tag)
                # print('New pattern: ', pattern_with_candidate)
                inner_delta = delta_tag
                inner_pattern = pattern_with_candidate
        if inner_delta < 0 or inner_delta < delta:
            # print("continue")
            continue
        else:
            # print('New delta: ', delta_tag)
            # print('New pattern: ', pattern_with_candidate)
            delta = inner_delta
            pattern = inner_pattern
    # print('Merging sequences: ', seq1[0], 'and', seq2[0])
    # print('with pattern: ', pattern)
            
    # print('in: ', seq1, seq2)
    # print('out ', delta, pattern)
    combined = [list(item) for item in set(tuple(row) for row in seq1[1] + seq2[1])]
    return delta, (pattern, combined)

In [28]:
def sortCandidates(x, seq1, seq2):
    return seq1[0].count(x) + seq2[0].count(x)

In [29]:
def notContainedEventsWithIndex(sequence: Sequence, pattern: Pattern) -> List[Tuple[int, str]]:
    output: List[Tuple[int, str]] = []
    for i in range(len(sequence)):
        if sequence[i] not in pattern:
            output.append((i, sequence[i]))
    return output

In [30]:
def longestCommonSubsquence(seq1: Pattern, seq2: Pattern) -> Pattern:
    # Resulting sequence does not need to be consecutive like a substring, but it does need to be in order
    if len(seq1) == 0 or len(seq2) == 0:
        return []

    matrix = [[[] for x in range(len(seq2))] for x in range(len(seq1))]
    for i in range(len(seq1)):
        for j in range(len(seq2)):
            if seq1[i] == seq2[j]:
                if i == 0 or j == 0:
                    matrix[i][j] = [seq1[i]]
                else:
                    copy = matrix[i-1][j-1].copy()
                    copy.append(seq1[i])
                    matrix[i][j] = copy
            else:
                matrix[i][j] = max(matrix[i-1][j], matrix[i][j-1], key=len)

    cs = matrix[-1][-1]
    
    return cs

In [31]:
def edits(seq: Sequence, pattern: Pattern) -> int:
    table = np.zeros((len(seq) + 1, len(pattern) + 1))
    for index, value in np.ndenumerate(table):
        if index[0] == 0 or index[1] == 0:
            table[index] = index[1] if index[0] == 0 else index[0]
        else:
            matching = 0 if seq[index[0]-1] == pattern[index[1]-1] else 1
            table[index] = min(table[(index[0] - 1, index[1])] + 1, table[(index[0], index[1] - 1)] + 1, table[(index[0] - 1, index[1] - 1)] + matching)
    
    return table[-1, -1]

print(edits(['i', 'n', 't', 'e', 'n','t','i','o','n'], ['e', 'x', 'e', 'c', 'u', 't', 'i', 'o', 'n']))

5.0


In [32]:
import pandas as pd

df = pd.read_csv('../public/data/nobel.csv')
grouped = df.groupby(['firstname', 'lastname'])
nobel_data = grouped['eventtype'].apply(list)
min_data = minDL(nobel_data)

Initialization phase over
Queue is empty


In [33]:
print('Number of patterns: ', len(min_data))
print('Number of sequences: ', sum([len(x[1]) for x in min_data]))
print([pattern[0] for pattern in min_data])
min_data

Number of patterns:  6
Number of sequences:  24
[['Birth', 'Marriage', 'Nobel Prize', 'Marriage', 'Death'], ['Birth', 'Undergraduate', 'Postgraduate', 'Doctorate', 'Marriage', 'Professorship', 'Nobel Prize', 'Death'], ['Birth', 'Abroad', 'Abroad', 'Abroad', 'Marriage', 'Professorship', 'Nobel Prize', 'Death'], ['Birth', 'Doctorate', 'Marriage', 'Professorship', 'Nobel Prize', 'Death'], ['Birth', 'Postgraduate', 'Doctorate', 'Professorship', 'Marriage', 'Nobel Prize', 'Death'], ['Birth', 'Undergraduate', 'Abroad', 'Postgraduate', 'Doctorate', 'Professorship', 'Marriage', 'Nobel Prize', 'Death']]


[(['Birth', 'Marriage', 'Nobel Prize', 'Marriage', 'Death'],
  [['Birth', 'Marriage', 'Nobel Prize', 'Marriage', 'Death']]),
 (['Birth',
   'Undergraduate',
   'Postgraduate',
   'Doctorate',
   'Marriage',
   'Professorship',
   'Nobel Prize',
   'Death'],
  [['Birth',
    'Abroad',
    'Undergraduate',
    'Postgraduate',
    'Marriage',
    'Doctorate',
    'Nobel Prize',
    'Professorship',
    'Nobel Prize',
    'Death'],
   ['Birth',
    'Undergraduate',
    'Postgraduate',
    'Marriage',
    'Professorship',
    'Nobel Prize',
    'Death'],
   ['Birth',
    'Undergraduate',
    'Marriage',
    'Doctorate',
    'Marriage',
    'Professorship',
    'Nobel Prize',
    'Death'],
   ['Birth',
    'Undergraduate',
    'Postgraduate',
    'Doctorate',
    'Marriage',
    'Professorship',
    'Nobel Prize',
    'Death']]),
 (['Birth',
   'Abroad',
   'Abroad',
   'Abroad',
   'Marriage',
   'Professorship',
   'Nobel Prize',
   'Death'],
  [['Birth',
    'Postgraduate',
    'Abroad',


In [34]:
import pandas as pd

df = pd.read_json('../public/data/events.json').head(500)
df['eventtype'] = df['type'].apply(lambda x: x['name'])
# df
grouped = df.groupby(['possession'])
events_data = grouped['eventtype'].apply(list)
min_events_data = minDL(events_data)

Initialization phase over
Queue is empty


In [36]:
print('Number of patterns: ', len(min_events_data))
print('Number of sequences: ', sum([len(x[1]) for x in min_events_data]))
print([pattern[0] for pattern in min_events_data])
min_events_data

Number of patterns:  17
Number of sequences:  29
[['Starting XI', 'Starting XI', 'Half Start', 'Half Start'], ['Camera On', 'Pressure', 'Pass', 'Ball Receipt*', 'Carry', 'Pass', 'Ball Receipt*', 'Pass', 'Pressure', 'Ball Receipt*', 'Miscontrol', 'Pressure', 'Ball Recovery', 'Carry', 'Foul Committed', 'Foul Won'], ['Pass', 'Pressure', 'Ball Receipt*', 'Interception', 'Carry', 'Miscontrol', 'Ball Recovery', 'Carry', 'Pressure', 'Pass'], ['Pass', 'Ball Receipt*', 'Pass', 'Ball Receipt*', 'Pass'], ['Pass', 'Clearance', 'Ball Recovery', 'Carry', 'Pressure', 'Pass', 'Block', 'Block'], ['Ball Recovery', 'Carry', 'Foul Committed', 'Foul Won'], ['Pass', 'Ball Receipt*', 'Pass', 'Pressure', 'Ball Receipt*', 'Pass', 'Pressure', 'Ball Receipt*', 'Interception'], ['Pass', 'Ball Receipt*', 'Carry', 'Pressure', 'Ball Receipt*', 'Carry', 'Pass', 'Ball Receipt*', 'Carry', 'Pressure', 'Pass', 'Pressure', 'Ball Receipt*', 'Carry', 'Ball Recovery', 'Pass', 'Ball Receipt*', 'Duel', 'Pass', 'Pass', 'Ball Re

[(['Starting XI', 'Starting XI', 'Half Start', 'Half Start'],
  [['Starting XI', 'Starting XI', 'Half Start', 'Half Start']]),
 (['Camera On',
   'Pressure',
   'Pass',
   'Ball Receipt*',
   'Carry',
   'Pass',
   'Ball Receipt*',
   'Pass',
   'Pressure',
   'Ball Receipt*',
   'Miscontrol',
   'Pressure',
   'Ball Recovery',
   'Carry',
   'Foul Committed',
   'Foul Won'],
  [['Camera On',
    'Pressure',
    'Pass',
    'Ball Receipt*',
    'Carry',
    'Pass',
    'Ball Receipt*',
    'Pass',
    'Pressure',
    'Ball Receipt*',
    'Miscontrol',
    'Pressure',
    'Ball Recovery',
    'Carry',
    'Foul Committed',
    'Foul Won']]),
 (['Pass',
   'Pressure',
   'Ball Receipt*',
   'Interception',
   'Carry',
   'Miscontrol',
   'Ball Recovery',
   'Carry',
   'Pressure',
   'Pass'],
  [['Pass',
    'Pressure',
    'Ball Receipt*',
    'Interception',
    'Carry',
    'Miscontrol',
    'Ball Recovery',
    'Carry',
    'Pressure',
    'Pass']]),
 (['Pass', 'Ball Receipt*', 'Pass